## Import

In [45]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV


from sklearn import tree
from sklearn.ensemble import RandomForestClassifier



## Load data

In [2]:
data = pd.read_csv('features.csv')
# Delete unesessary features from dataset
data = data.dropna()
#data = data.drop(['enterworld_num','buyitemnowmainauction_num','completechallengeweek_num'],axis=1)
data = data.drop(['actor_account_id'
                    ,'survival_time'
                    ],axis=1)
data = data.drop(columns=data.columns[0],axis=1)
# Setup & split dataset for training
X = data.copy()
y = X.pop('churn_yn')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)
# Print training data shape
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)
# Get feature names from training set
feature_names = [f"feature {i}" for i in range(X.shape[1])]

(995, 36) (995,)
(2985, 36) (2985,)


In [3]:
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)
predictions = forest.predict(X_test)

In [4]:
#F1-score
print("F1 Score")
print('f1 ' , f1_score(y_test, predictions))
#Cross validation
scores = cross_val_score(forest, X, y, cv=5)
print("CrossValidation Scores: ")
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

F1 Score
f1  0.5784499054820417
CrossValidation Scores: 
[0.78015075 0.74371859 0.79271357 0.75125628 0.7701005 ]
0.77 accuracy with a standard deviation of 0.02


In [5]:
result = permutation_importance(
    forest, X_test, y_test, n_repeats=10, random_state=100, n_jobs=2)

In [6]:
num = 5
feat_importances_p = pd.DataFrame(result.importances_mean, index=forest.feature_names_in_, columns=["Importance"])
feat_importances_p['index1'] = feat_importances_p.index
top = feat_importances_p['index1'].iloc[:num].values
flop = feat_importances_p['index1'].iloc[-num:].values
a="churn_yn"
topflop = np.append(top,flop)
topflop = np.append(topflop,a)
print(topflop)


['masteryexp' 'enterworld_num' 'longest_time_between_events'
 'average_money_spent_per_session' 'targetaccountid_num'
 'itemupgrade_successrate' 'levelup_num' 'reason_spendmoney'
 'partybattles_per_session' 'joinparty_num' 'churn_yn']


In [7]:
data = pd.read_csv('features.csv',usecols= topflop)
data = data.dropna()
print(data)
# Setup & split dataset for training
X = data.copy()
y = X.pop('churn_yn')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)
# Print training data shape
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)
# Get feature names from training set
feature_names = [f"feature {i}" for i in range(X.shape[1])]

      churn_yn  enterworld_num  levelup_num  joinparty_num  \
1          0.0           193.0          5.0          176.0   
2          0.0           176.0         19.0          148.0   
3          0.0           102.0         74.0          104.0   
4          0.0           237.0          0.0          273.0   
5          1.0            28.0          0.0           12.0   
...        ...             ...          ...            ...   
4015       1.0           176.0         39.0           54.0   
4016       1.0             6.0          5.0            0.0   
4017       0.0           122.0        106.0            4.0   
4018       0.0           894.0          6.0          410.0   
4019       0.0           322.0          0.0          255.0   

      itemupgrade_successrate  longest_time_between_events  \
1                        0.00                    253529.85   
2                        0.00                    436143.07   
3                        0.00                    458000.84   
4      

In [8]:
param_grid = { 
    'n_estimators': [1000, 1200],
    'max_features': ['sqrt','log2'],
    'max_depth' : [4],
    'criterion' :['gini']
}

In [9]:
forest = RandomForestClassifier(random_state=0)
CV_rfc = GridSearchCV(estimator=forest,param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'criterion': ['gini'], 'max_depth': [4],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [1000, 1200]})

In [10]:
print(CV_rfc.best_params_)
param_grid=CV_rfc.best_params_

{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 1000}


In [43]:
forest = RandomForestClassifier(random_state=0,criterion='gini', max_depth=7,max_features='sqrt',n_estimators=100)
forest.fit(X_train, y_train)
predictions = forest.predict(X_test)

In [44]:
#F1-score
print("F1 Score")
print('f1 ' , f1_score(y_test, predictions))
#Cross validation
scores = cross_val_score(forest, X, y, cv=5)
print("CrossValidation Scores: ")
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

F1 Score
f1  0.5925925925925926
CrossValidation Scores: 
[0.78768844 0.74497487 0.79020101 0.75502513 0.7638191 ]
0.77 accuracy with a standard deviation of 0.02


In [47]:
filename = 'forest_model.sav'
pickle.dump(forest, open(filename, 'wb'))